# Building a Chatbot with Conversation History

In this notebook, we will build a chatbot powered by a large language model (LLM). The main goal is to create a chatbot that can:

- **Understand user messages** and respond naturally.
- **Remember previous interactions** using a conversation history mechanism.
- **Support multi-turn conversations**, so the chatbot can refer back to earlier messages.

## Key Concepts Covered

### 1. LLM Integration
We use `ChatGroq` from LangChain to generate responses based on user input.

### 2. Message History
By storing all previous messages in a session, the chatbot can maintain context across multiple turns.

### 3. Prompt Templates
We define system instructions and message placeholders to structure user input for the model.

### 4. Conversation Management
We trim and manage message history to prevent the context from becoming too large. 



In [14]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
#groq_api_key



In [15]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
# model

In [16]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Nidhi and I am a AI Engineer")])

AIMessage(content="Nice to meet you Nidhi, a AI Engineer. That's a fascinating field, and I'm excited to learn more about your work. What specific areas of AI do you specialize in, such as computer vision, natural language processing, or machine learning?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 49, 'total_tokens': 102, 'completion_time': 0.086349076, 'prompt_time': 0.002720596, 'queue_time': 0.049058224, 'total_time': 0.089069672}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--fd9f8588-0767-46a7-9f15-55f8c37eba2b-0', usage_metadata={'input_tokens': 49, 'output_tokens': 53, 'total_tokens': 102})

In [17]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Nidhi and I am a  AI Engineer"),
        AIMessage(content="Hello Nidhi! It's nice to meet you. \n\nAs a Nidhi AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="Your name is Nidhi, and you're an AI Engineer. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 123, 'total_tokens': 146, 'completion_time': 0.026586336, 'prompt_time': 0.006744916, 'queue_time': 0.049342584, 'total_time': 0.033331252}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6b6db12a-13a1-4301-8d1c-ee7f1a96c2df-0', usage_metadata={'input_tokens': 123, 'output_tokens': 23, 'total_tokens': 146})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [18]:
!pip install langchain_community


[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [20]:
config={"configurable":{"session_id":"chat1"}}

In [21]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Nidhi and I am a  AI Engineer")],
    config=config
)

In [22]:
response.content

'Nice to meet you, Nidhi. As an AI Engineer, you must be working on exciting projects that involve the development of intelligent systems. What specific areas of AI are you interested in or currently working on? Are you into machine learning, natural language processing, computer vision, or something else?'

In [23]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="Your name is Nidhi. You're a AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 125, 'total_tokens': 139, 'completion_time': 0.026916571, 'prompt_time': 0.006819226, 'queue_time': 0.052284494, 'total_time': 0.033735797}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2c05a815-bae9-44c1-b7b5-edc4c97eaaaa-0', usage_metadata={'input_tokens': 125, 'output_tokens': 14, 'total_tokens': 139})

In [24]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"I don't have any information about your name. I'm a large language model, I don't have the ability to retain personal information about individual users. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. If you'd like to share your name with me, I'd be happy to chat with you!"

In [25]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Nidhi")],
    config=config1
)
response.content

"Nice to meet you, Nidhi. How's your day going so far?"

In [26]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Nidhi.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [27]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [28]:
chain.invoke({"messages":[HumanMessage(content="Hey My name is Nidhi")]
              
              })

AIMessage(content="Nice to meet you, Nidhi! I'm happy to be your assistant. Is there anything I can help you with today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 59, 'total_tokens': 87, 'completion_time': 0.044975186, 'prompt_time': 0.002849714, 'queue_time': 0.049307416, 'total_time': 0.0478249}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--bb244ce3-3748-4373-bbb4-7e18e4109055-0', usage_metadata={'input_tokens': 59, 'output_tokens': 28, 'total_tokens': 87})

In [29]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "chat3"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hey My name is Nidhi")],
    config = config)

In [30]:
response

AIMessage(content="Hello Nidhi, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 59, 'total_tokens': 87, 'completion_time': 0.036662669, 'prompt_time': 0.002844534, 'queue_time': 0.052160056, 'total_time': 0.039507203}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--869ce4db-e83a-4ad2-a958-eab6c552307b-0', usage_metadata={'input_tokens': 59, 'output_tokens': 28, 'total_tokens': 87})

In [31]:
response = with_message_history.invoke(
    [HumanMessage(content = "What is My name?")],
    config= config
)

In [32]:
response.content

'Your name is Nidhi.'

In [33]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [34]:
response = chain.invoke({"messages":[HumanMessage(content="Hey My name is Nidhi")],"language": "Hindi"})
response.content

'नमस्ते निधि, मैं आपकी सहायता करने के लिए यहाँ हूँ। क्या मैं आपकी किसी समस्या या प्रश्न का समाधान कर सकता हूँ?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [35]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [36]:
config ={ "configurable": {"session_id":"chat4"}}
response = with_message_history.invoke({"messages":[HumanMessage(content="Hi i am nidhi")], "language" :"Hindi"},
                                       config = config)


In [37]:
response.content

'नमस्ते निधि! मैं आपकी सहायता करने के लिए यहाँ हूँ। क्या आपके पास कोई प्रश्न है या आपकी कोई समस्या है जिसका समाधान चाहती हैं?'

In [38]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What is My name?")], "language" :"Hindi"},
                                       config = config)
    
    


In [39]:
response.content

'आपका नाम निधि है।'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [40]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [41]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"Unfortunately, I'm a large language model, I don't have any information about your personal preferences. You'll have to tell me what your favorite ice cream flavor is!"

In [42]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked me to solve the math problem "2 + 2".'

In [43]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [44]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"You didn't tell me your name, so I don't know it. Would you like to share your name with me?"

In [45]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't asked a math problem yet. Our conversation just started, and I'm ready to help with any question you may have. What's the first problem or question you'd like to discuss?"